In [93]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'
import openai
import string
import numpy as np
import pandas as pd
from gpt_index import (
    GPTKeywordTableIndex,
    GPTSimpleKeywordTableIndex,
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper,
    GPTSimpleVectorIndex,
    GPTListIndex,
    readers,
    KeywordExtractPrompt,
    QuestionAnswerPrompt
)
from langchain import OpenAI, PromptTemplate

In [2]:
path2file = r'C:\Users\aa\Desktop\University\Semester 5\DSSG\DRK\scraped.csv'
df = pd.read_csv(path2file)

In [23]:
#df_texts = df['text']
df_texts = df[df['text'].str.contains('Obdachlosigkeit')].reset_index()

In [32]:
df_texts = df_texts.text.tolist()

In [33]:
# getting only the relevant documents with initial filteration
for i in range(len(texts_keywords)):
    filepath = os.path.join(r'C:\Users\aa\Desktop\University\Semester 5\DSSG\DRK\text_keywords',f'{texts_keywords[i]}.txt')
    with open(filepath, 'w',encoding="utf-8") as f:
        f.write(texts_[texts_keywords[i]])

In [74]:
def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens (default)
    num_outputs = 256
    # maximum chunk overlap
    chunk_overlap = 20 

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name='text-davinci-002', max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_overlap, chunk_size_limit=512)
    
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    # Using Simple Vector Index with default embeddings
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index3.json')

    return index

In [80]:
# text_files_test has 30 files for indexing
path_files = r'C:\Users\aa\Desktop\University\Semester 5\DSSG\DRK\text_files_test'
index = construct_index(path_files)

> Adding chunk: Kinder sind unsere Zukunft â€“ in der Gegenwart...
> Adding chunk: Wir betreuen Ihre Kinder von Montag bis Freitag...
> Adding chunk: Herzlich willkommen in unserer integrativen DRK...
> Adding chunk: Erziehungsberatung sowie mit dem Landratsamt un...
> Adding chunk: Wir betreuen Ihre Kinder von Montag bis Freitag...
> Adding chunk: In der Kleiderkammer des DRK Kreisverbandes Got...
> Adding chunk: Wir haben das Ziel, durch individuelle Betreuun...
> Adding chunk: noch stÃ¤rker als bisher in den Fokus unserer B...
> Adding chunk: Sie selbst, Ihr Partner, ein Elternteil oder ei...
> Adding chunk: Mitarbeiter fÃ¼r die Hauswirtschaft in der ambu...
> Adding chunk: dem daraus resultierenden Menschenbild,
â€¢ ein...
> Adding chunk: Der DRK Kreisverband Gotha e.V. ist TrÃ¤ger von...
> Adding chunk: noch erworben werden)

- Kommunikative FÃ¤higke...
> Adding chunk: FÃ¼r unsere DRK- KindertagesstÃ¤tte â€žPittipla...
> Adding chunk: mit den Rot-Kreuz-GrundsÃ¤tzen und dem daraus 

In [140]:
index = GPTSimpleVectorIndex.load_from_disk('index3.json')

In [142]:
def ask_texts():
    RETRIEVE_DOCS = 0
    SUMMARIZE_DOCS = 1

    state = RETRIEVE_DOCS


    if (state == RETRIEVE_DOCS):
        query = input("Welche Art von Dokumente wollen Sie?")

#             template = """
#             Ich gebe dir Texte und will wissen zu welcher der folgenden Keywords es passt: {keywords}.
#             Wenn es zu keinem der Keywords passt gibt mir 'irrelevant' zurück.
#             """

#             prompt = PromptTemplate(
#                 input_variables=["keywords"],
#                 template=template,
#             )

#             prompt1 = QuestionAnswerPrompt(langchain_prompt=prompt)

        # Default Prompt Template was used with a specified similarity_top_k filter
        response = index.query(query, similarity_top_k=10, verbose=True, response_mode='default')
        retrieved_docs = []
        for sn in response.source_nodes:
            retrieved_docs.append(readers.Document(sn.source_text))
    return retrieved_docs
                
#             state = SUMMARIZE_DOCS
            
            index2 = GPTListIndex(retrieved_docs)
            
#             while state == SUMMARIZE_DOCS:
#                 query = input(" \033[1m Was würden Sie gerne dazu wissen wollen? \033[0m")
                
#                 if(query == 'exit'):
#                     state = RETRIEVE_DOCS
#                     break
                
#                 response = index2.query(query, response_mode="compact", verbose=True)
#                 print(f"\033[1m {response.response} \033[0m")

In [ ]:
retrieved = ask_texts()
retrieved

Welche Art von Dokumente wollen Sie?Alle Dokumente zu Kleiderklammer für Obdachlose
> Top 10 nodes:
> [Node 1e61e4cc-afb3-40b1-a9bd-5c2b9acbd568] [Similarity score:                     0.851841] In der Kleiderkammer des DRK Kreisverbandes Gotha e.V. versorgen wir bedÃ¼rfte oder in Not gerate...
> [Node 986b6ecd-17e6-4788-98fc-6d75664a420c] [Similarity score:                     0.832788] die Mitarbeiterin Nelly Engelmann. â€žDenn die Altkleider werden alle von uns per Hand gesichtet....
> [Node f3b56dab-0880-450f-bb19-b000a2aa5665] [Similarity score:                     0.830595] dem Katastrophenschutz sowie dem Kriseninterventionsteam.

â€žAltkleider gehÃ¶ren nicht in den MÃ...
> [Node 3ff2d89e-8099-4a8d-8c95-b5d5463b7c7c] [Similarity score:                     0.82879] sind.

Der Kleiderladen hat von Montag bis Freitag geÃ¶ffnet. Dann herrscht dort reger Betrieb. V...
> [Node 8eb183e1-338f-45a1-9c4c-83e08c542579] [Similarity score:                     0.820514] â€žDer wirtschaftliche

> Refined response: 

Alle Dokumente zu Kleiderkammern für Obdachlose sind beim DRK Kreisverband Gotha e.V. erhältlich. Dort können Sie sich über die Kleiderkammer informieren und auch über die Mitarbeiterin Nelly Engelmann, die die Altkleider per Hand sichtet. Der Kleiderladen steht allen Personengruppen offen, egal ob Bedürftige, Studenten oder Azubis. Auch Haushaltsutensilien wie Gläser und Porzellan können hier gekauft werden. Nicht mehr tragbare und beschädigte Textilien werden an eine Verwertungsgesellschaft weitergegeben, deren Einnahmen direkt in die Rotkreuzarbeit vor Ort, einschließlich dem Katastrophenschutz sowie dem Kriseninterventionsteam, fließen. 

Das DRK betreibt die Le
> Refine context: schon von unseren Mitarbeitern entsorgt werden....
> Refined response: 

Alle Dokumente zu Kleiderkammern für Obdachlose sind beim DRK Kreisverband Gotha e.V. erhältlich. Dort können Sie sich über die Kleiderkammer informieren und auch über die Mitarbeiterin Nelly Engelmann, die die A

# Future Steps
- top similarity will be set 70% and above similarity
- An automized query and not a user input. User Input was used for easier testing
- Accuracy for the results on a larger test sample n =< 50?



In [122]:
# a different approach using the GPTKeywordTableIndex. Still being developed
def ask_texts_key():
    RETRIEVE_DOCS = 0
    SUMMARIZE_DOCS = 1

    state = RETRIEVE_DOCS

    while True:
        if (state == RETRIEVE_DOCS):
            query = input("Welche Art von Dokumente wollen Sie?")
            
            response = index.query(query, similarity_top_k=10, verbose=True, response_mode='default')
            retrieved_docs = []
            for sn in response.source_nodes:
                retrieved_docs.append(readers.Document(sn.source_text))
                
            state = SUMMARIZE_DOCS
            
            index2 = GPTKeywordTableIndex(retrieved_docs)
            
            while state == SUMMARIZE_DOCS:
                query = input(" \033[1m Was würden Sie gerne dazu wissen wollen? \033[0m")
                
                if(query == 'exit'):
                    state = RETRIEVE_DOCS
                    break
                
                response = index2.query(query, response_mode="compact", verbose=True)
                print(f"\033[1m {response.response} \033[0m")